In [2]:
from keras.layers import Conv2D  
from keras.layers import Convolution2D 
from keras.models import Sequential  
from keras.layers import Dense, Dropout, Flatten, MaxPooling2D, GlobalAveragePooling2D, Activation  
from keras.layers import Conv2D
from keras.models import Sequential  


In [3]:
IMG_SAVE_PATH = r'D:\Project\train' 

In [4]:
Str_to_Int = {
    'Acne and Rosacea Photos': 0,
    'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions': 1,
    'Atopic Dermatitis Photos': 2,
    'Benign Keratosis Photos': 3,
    'Cellulitis Impetigo and other Bacterial Infections': 4,
    'Eczema Photos': 5,
    'Exanthems and Drug Eruptions': 6,
    'Hair Loss Photos Alopecia and other Hair Diseases': 7,
    'Herpes HPV and other STDs Photos': 8,
    'Light Diseases and Disorders of Pigmentation': 9,
    'Lupus and other Connective Tissue diseases': 10,
    'Melanoma Skin Cancer Nevi and Moles': 11,
    'Nail Fungus and other Nail Disease': 12,
    'Poison Ivy Photos and other Contact Dermatitis': 13,
    'Psoriasis pictures Lichen Planus and related diseases': 14,
    'Scabies Lyme Disease and other Infestations and Bites': 15,
    'Seborrheic Keratoses and other Benign Tumors': 16,
    'Systemic Disease': 17,
    'Tinea Ringworm Candidiasis and other Fungal Infections': 18,
    'Urticaria Hives': 19,
    'Vascular Tumors': 20,
    'Viral Infections': 21,
    'Warts Molluscum and other Viral Infections': 22
}

NUM_CLASSES = 23

def str_to_Int_mapper(val):
    return Str_to_Int[val]

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    IMG_SAVE_PATH,
    target_size=(240, 240),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    IMG_SAVE_PATH,
    target_size=(240, 240),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 12453 images belonging to 23 classes.
Found 3104 images belonging to 23 classes.


In [6]:
#this changes into class_mode categorical
# def to_categorical_manual(labels, num_classes):
#     one_hot = []
#     for label in labels:
#         vec = [0] * num_classes
#         vec[label] = 1
#         one_hot.append(vec)
#     return one_hot

In [7]:
count = 0

# Check first N batches to verify shapes
for batch in train_generator:
    images, labels = batch
    for img in images:
        if img.shape != (240, 240, 3):
            count += 1
    break  # Remove this `break` if you want to check all batches

print("Count of images with wrong shape:", count)


Count of images with wrong shape: 0


# Dense Net

In [8]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

# Load DenseNet121 pretrained on ImageNet, exclude the top classification layer
densenet = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(240, 240, 3)
)

def build_densenet():
    model = Sequential()
    model.add(densenet)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(NUM_CLASSES, activation='softmax'))  # NUM_CLASSES = 23

    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(learning_rate=0.00005),
        metrics=['accuracy']
    )
    
    return model


In [ ]:
# Import required libraries
import seaborn as sns
from matplotlib import pyplot

# Define function to plot training & validation accuracy and loss
def plot_acc(history):
    # Set seaborn style for better visual aesthetics
    sns.set()

    # Create a figure with width=12, height=4
    fig = pyplot.figure(0, (12, 4))

    # === Plot Accuracy ===
    ax = pyplot.subplot(1, 2, 1)
    sns.lineplot(
        x=history.epoch,
        y=history.history['accuracy'],
        label='Train Accuracy'
    )
    sns.lineplot(
        x=history.epoch,
        y=history.history['val_accuracy'],
        label='Validation Accuracy'
    )
    pyplot.title('Model Accuracy')
    pyplot.xlabel('Epochs')
    pyplot.ylabel('Accuracy')
    pyplot.legend()
    pyplot.tight_layout()

    # === Plot Loss ===
    ax = pyplot.subplot(1, 2, 2)
    sns.lineplot(
        x=history.epoch,
        y=history.history['loss'],
        label='Train Loss'
    )
    sns.lineplot(
        x=history.epoch,
        y=history.history['val_loss'],
        label='Validation Loss'
    )
    pyplot.title('Model Loss')
    pyplot.xlabel('Epochs')
    pyplot.ylabel('Loss')
    pyplot.legend()
    pyplot.tight_layout()



In [ ]:
    # Display the plots
    pyplot.show()

In [ ]:
# Build model
model = build_densenet()

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Plot accuracy and loss
plot_acc(history)


Epoch 1/10


C:\Users\Sandesh Khatiwada\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 72/390 ━━━━━━━━━━━━━━━━━━━━ 2:35:46 29s/step - accuracy: 0.0687 - loss: 3.9510